In [5]:
import pandas as pd  

df = pd.read_csv('Sample Game.csv') 

print (df) 

players = ['A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5'] 

plus_minus_dict = {player : 0 for player in players} 

   Player A1 Player A2 Player A3 Player B1 Player B2 Player B3  Team A Points  \
0         A1        A2        A3        B1        B2        B3             11   
1         A1        A2        A3        B1        B2        B4              2   
2         A1        A4        A5        B1        B4        B5              7   
3         A1        A4        A5        B3        B4        B5              6   
4         A2        A3        A4        B2        B3        B4              4   
5         A2        A3        A5        B2        B3        B5              5   
6         A1        A2        A3        B2        B4        B5              4   
7         A1        A2        A3        B1        B4        B5              4   
8         A3        A4        A5        B1        B4        B5              4   
9         A1        A3        A4        B1        B2        B3              5   
10        A1        A4        A5        B2        B3        B4              2   

    Team B Points  possessi

In [6]:
for player in players: 
    # find rows where they appear 
    # find the point differential between A and B 
    # add that to their plus minus -- positive if an A player, negative if a B player 
    for index in range(len(df)):  
        if player in df.loc[index, :].values: 
            differential = df.loc[index, "Team A Points"] - df.loc[index, "Team B Points"] 
            if "B" in player: 
                differential *= -1 
            plus_minus_dict[player] += differential 
    
print(plus_minus_dict) 

{'A1': 2, 'A2': -5, 'A3': 2, 'A4': 3, 'A5': 1, 'B1': -14, 'B2': 6, 'B3': 3, 'B4': 6, 'B5': -4}


In [7]:
import numpy as np 

df.insert(loc=len(df), column='Differential per 100 poss', value=np.zeros(len(df))) 

for index in range(len(df)):  
    # Insert the normalized differential per 100 possessions 
    differential = df.loc[index, "Team A Points"] - df.loc[index, "Team B Points"]  
    poss = df.loc[index, "possessions"] 
    diff_per_100 = differential * 100/poss 
    df.loc[index, "Differential per 100 poss"] = diff_per_100 

player_matrix = np.zeros((len(df), len(players)))

for i in range(len(players)): 
    player = players[i]
    for index in range(len(df)):  
        if player in df.loc[index, :].values: 
            value = 1
            if "B" in player: 
                value *= -1 
            player_matrix[index, i] = value 
    
beta_vec = np.zeros(len(players)) 

diffs_vec = np.array(df.loc[:, "Differential per 100 poss"]) 

new_X = player_matrix.T @ player_matrix 

new_y = player_matrix.T @ diffs_vec

adjusted_plus_minus_vec = np.linalg.inv(new_X) @ new_y 

APM_vec_per_poss = adjusted_plus_minus_vec / 100 

print(APM_vec_per_poss) 

[-0.32 -0.64 -0.96  0.   -0.96 -1.28 -0.32 -0.96  0.   -0.96]
